## Test environment

In [1]:
import primer3
import pandas as pd

from IPython.display import display

# Configure Jupyter to show scrollable output
%config InlineBackend.print_figure_kwargs = {'bbox_inches': None}


# Import modules
from multiplexdesigner.utils import MultiplexPanel

In [2]:
def create_primer_database(data, db_name='primer_results.db'):
    """
    Convert primer design results to a SQLite database with proper schema
    """
    
    # Connect to SQLite database (creates if doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create tables with proper schema
    create_tables(cursor)
    
    # Insert the data
    analysis_id = insert_analysis_summary(cursor, data)
    insert_primer_pairs(cursor, data, analysis_id)
    insert_primers(cursor, data, analysis_id)
    insert_internal_primers(cursor, data, analysis_id)
    
    # Commit changes and close
    conn.commit()
    conn.close()
    
    print(f"Database created successfully: {db_name}")
    return db_name

def create_tables(cursor):
    """Create all necessary tables with proper relationships"""
    
    # Analysis summary table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS analysis_summary (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            created_date TEXT DEFAULT CURRENT_TIMESTAMP,
            left_primers_considered INTEGER,
            right_primers_considered INTEGER,
            internal_primers_considered INTEGER,
            pairs_considered INTEGER,
            left_primers_returned INTEGER,
            right_primers_returned INTEGER,
            internal_primers_returned INTEGER,
            pairs_returned INTEGER,
            left_explain TEXT,
            right_explain TEXT,
            internal_explain TEXT,
            pair_explain TEXT
        )
    ''')
    
    # Primer pairs table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS primer_pairs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            pair_rank INTEGER,
            penalty REAL,
            compl_any_th REAL,
            compl_end_th REAL,
            product_size INTEGER,
            product_tm REAL,
            left_primer_id INTEGER,
            right_primer_id INTEGER,
            internal_primer_id INTEGER,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Left primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS left_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            end_stability REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Right primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS right_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            end_stability REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')
    
    # Internal primers table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS internal_primers (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            analysis_id INTEGER,
            primer_rank INTEGER,
            sequence TEXT,
            start_pos INTEGER,
            length INTEGER,
            penalty REAL,
            tm REAL,
            gc_percent REAL,
            self_any_th REAL,
            self_end_th REAL,
            hairpin_th REAL,
            FOREIGN KEY (analysis_id) REFERENCES analysis_summary (id)
        )
    ''')

def insert_analysis_summary(cursor, data):
    """Insert analysis summary and return the analysis_id"""
    cursor.execute('''
        INSERT INTO analysis_summary (
            left_primers_returned, right_primers_returned, 
            internal_primers_returned, pairs_returned,
            left_explain, right_explain, internal_explain, pair_explain
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        data.get('PRIMER_LEFT_NUM_RETURNED', 0),
        data.get('PRIMER_RIGHT_NUM_RETURNED', 0),
        data.get('PRIMER_INTERNAL_NUM_RETURNED', 0),
        data.get('PRIMER_PAIR_NUM_RETURNED', 0),
        data.get('PRIMER_LEFT_EXPLAIN', ''),
        data.get('PRIMER_RIGHT_EXPLAIN', ''),
        data.get('PRIMER_INTERNAL_EXPLAIN', ''),
        data.get('PRIMER_PAIR_EXPLAIN', '')
    ))
    return cursor.lastrowid

def insert_primer_pairs(cursor, data, analysis_id):
    """Insert primer pair data"""
    pairs = data.get('PRIMER_PAIR', [])
    
    for rank, pair in enumerate(pairs):
        cursor.execute('''
            INSERT INTO primer_pairs (
                analysis_id, pair_rank, penalty, compl_any_th, compl_end_th,
                product_size, product_tm
            ) VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank, 
            pair.get('PENALTY', 0),
            pair.get('COMPL_ANY_TH', 0),
            pair.get('COMPL_END_TH', 0),
            pair.get('PRODUCT_SIZE', 0),
            pair.get('PRODUCT_TM', 0)
        ))

def insert_primers(cursor, data, analysis_id):
    """Insert left and right primer data"""
    
    # Left primers
    left_primers = data.get('PRIMER_LEFT', [])
    for rank, primer in enumerate(left_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO left_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th,
                hairpin_th, end_stability
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0),
            primer.get('END_STABILITY', 0)
        ))
    
    # Right primers
    right_primers = data.get('PRIMER_RIGHT', [])
    for rank, primer in enumerate(right_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO right_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th,
                hairpin_th, end_stability
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0),
            primer.get('END_STABILITY', 0)
        ))

def insert_internal_primers(cursor, data, analysis_id):
    """Insert internal primer data"""
    internal_primers = data.get('PRIMER_INTERNAL', [])
    
    for rank, primer in enumerate(internal_primers):
        coords = primer.get('COORDS', [0, 0])
        cursor.execute('''
            INSERT INTO internal_primers (
                analysis_id, primer_rank, sequence, start_pos, length,
                penalty, tm, gc_percent, self_any_th, self_end_th, hairpin_th
            ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            analysis_id, rank,
            primer.get('SEQUENCE', ''),
            coords[0] if len(coords) > 0 else 0,
            coords[1] if len(coords) > 1 else 0,
            primer.get('PENALTY', 0),
            primer.get('TM', 0),
            primer.get('GC_PERCENT', 0),
            primer.get('SELF_ANY_TH', 0),
            primer.get('SELF_END_TH', 0),
            primer.get('HAIRPIN_TH', 0)
        ))

def query_database(db_name='primer_results.db'):
    """Example queries to retrieve data from the database"""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    print("=== DATABASE QUERIES ===\n")
    
    # Query 1: Best primer pairs
    print("1. Best Primer Pairs (ordered by penalty):")
    cursor.execute('''
        SELECT pair_rank, penalty, product_size, product_tm, compl_any_th, compl_end_th
        FROM primer_pairs 
        ORDER BY penalty 
        LIMIT 5
    ''')
    
    for row in cursor.fetchall():
        print(f"   Rank {row[0]}: Penalty={row[1]:.3f}, Size={row[2]}bp, Tm={row[3]:.1f}°C")
    
    # Query 2: All primer sequences for best pair
    print("\n2. Primer Sequences for Best Pair:")
    cursor.execute('''
        SELECT l.sequence as left_seq, r.sequence as right_seq, i.sequence as internal_seq
        FROM primer_pairs pp
        JOIN left_primers l ON l.primer_rank = 0 AND l.analysis_id = pp.analysis_id
        JOIN right_primers r ON r.primer_rank = 0 AND r.analysis_id = pp.analysis_id  
        JOIN internal_primers i ON i.primer_rank = 0 AND i.analysis_id = pp.analysis_id
        WHERE pp.pair_rank = 0
    ''')
    
    result = cursor.fetchone()
    if result:
        print(f"   Left:     {result[0]}")
        print(f"   Right:    {result[1]}")
        print(f"   Internal: {result[2]}")
    
    # Query 3: Analysis summary
    print("\n3. Analysis Summary:")
    cursor.execute('SELECT * FROM analysis_summary ORDER BY id DESC LIMIT 1')
    summary = cursor.fetchone()
    if summary:
        print(f"   Left primers found: {summary[5]}")
        print(f"   Right primers found: {summary[6]}")
        print(f"   Primer pairs found: {summary[8]}")
    
    conn.close()

# Convert different sections to DataFrames
def convert_primer_data_to_tables(data):
    tables = {}
    
    # 1. Primer Pairs Summary
    if 'PRIMER_PAIR' in data:
        primer_pairs_df = pd.DataFrame(data['PRIMER_PAIR'])
        primer_pairs_df.index.name = 'Pair_ID'
        primer_pairs_df = primer_pairs_df.round(3)  # Round decimals
        tables['primer_pairs'] = primer_pairs_df
    
    # 2. Left Primers
    if 'PRIMER_LEFT' in data:
        left_primers_df = pd.DataFrame(data['PRIMER_LEFT'])
        left_primers_df.index.name = 'Left_Primer_ID'
        left_primers_df = left_primers_df.round(3)
        tables['left_primers'] = left_primers_df
    
    # 3. Right Primers  
    if 'PRIMER_RIGHT' in data:
        right_primers_df = pd.DataFrame(data['PRIMER_RIGHT'])
        right_primers_df.index.name = 'Right_Primer_ID'
        right_primers_df = right_primers_df.round(3)
        tables['right_primers'] = right_primers_df
    
    # 4. Internal Primers
    if 'PRIMER_INTERNAL' in data:
        internal_primers_df = pd.DataFrame(data['PRIMER_INTERNAL'])
        internal_primers_df.index.name = 'Internal_Primer_ID'
        internal_primers_df = internal_primers_df.round(3)
        tables['internal_primers'] = internal_primers_df
    
    # 5. Summary Statistics
    summary_data = {
        'Metric': ['Left Primers Returned', 'Right Primers Returned', 'Internal Primers Returned', 'Primer Pairs Returned'],
        'Count': [
            data.get('PRIMER_LEFT_NUM_RETURNED', 0),
            data.get('PRIMER_RIGHT_NUM_RETURNED', 0), 
            data.get('PRIMER_INTERNAL_NUM_RETURNED', 0),
            data.get('PRIMER_PAIR_NUM_RETURNED', 0)
        ]
    }
    tables['summary'] = pd.DataFrame(summary_data)
    
    return tables

def create_primer_dataframe(primer_data):
    """
    Convert primer design results to organized pandas DataFrames
    """
    
    # Method 1: Create a comprehensive primer pairs DataFrame
    primer_pairs = []
    
    num_pairs = primer_data.get('PRIMER_PAIR_NUM_RETURNED', 0)
    
    for i in range(num_pairs):
        pair_info = {
            'pair_id': i,
            'pair_penalty': primer_data.get(f'PRIMER_PAIR_{i}_PENALTY'),
            'product_size': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_SIZE'),
            'product_tm': primer_data.get(f'PRIMER_PAIR_{i}_PRODUCT_TM'),
            'compl_any_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_ANY_TH'),
            'compl_end_th': primer_data.get(f'PRIMER_PAIR_{i}_COMPL_END_TH'),
            
            # Left primer info
            'left_sequence': primer_data.get(f'PRIMER_LEFT_{i}_SEQUENCE'),
            'left_coords': primer_data.get(f'PRIMER_LEFT_{i}'),
            'left_tm': primer_data.get(f'PRIMER_LEFT_{i}_TM'),
            'left_gc_percent': primer_data.get(f'PRIMER_LEFT_{i}_GC_PERCENT'),
            'left_penalty': primer_data.get(f'PRIMER_LEFT_{i}_PENALTY'),
            'left_self_any_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_ANY_TH'),
            'left_self_end_th': primer_data.get(f'PRIMER_LEFT_{i}_SELF_END_TH'),
            'left_hairpin_th': primer_data.get(f'PRIMER_LEFT_{i}_HAIRPIN_TH'),
            'left_end_stability': primer_data.get(f'PRIMER_LEFT_{i}_END_STABILITY'),
            
            # Right primer info
            'right_sequence': primer_data.get(f'PRIMER_RIGHT_{i}_SEQUENCE'),
            'right_coords': primer_data.get(f'PRIMER_RIGHT_{i}'),
            'right_tm': primer_data.get(f'PRIMER_RIGHT_{i}_TM'),
            'right_gc_percent': primer_data.get(f'PRIMER_RIGHT_{i}_GC_PERCENT'),
            'right_penalty': primer_data.get(f'PRIMER_RIGHT_{i}_PENALTY'),
            'right_self_any_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_ANY_TH'),
            'right_self_end_th': primer_data.get(f'PRIMER_RIGHT_{i}_SELF_END_TH'),
            'right_hairpin_th': primer_data.get(f'PRIMER_RIGHT_{i}_HAIRPIN_TH'),
            'right_end_stability': primer_data.get(f'PRIMER_RIGHT_{i}_END_STABILITY'),
        }
        primer_pairs.append(pair_info)
    
    df_pairs = pd.DataFrame(primer_pairs)

    return(df_pairs)


In [3]:
# Get the mutation list (junctions) around which to design primer pairs
panel = MultiplexPanel("test_panel", "hg38")
panel.load_config(config_path="./config/designer_default_config.json")
panel.import_junctions_csv("./data/design_regions.csv")

Successfully imported 4 junctions from ./data/design_regions.csv


In [4]:
print(panel.junctions)

[Junction(EGFR_T790M, chr7:55181378-55181378), Junction(KRAS_G12D, chr12:25245350-25245350), Junction(KRAS_G13R, chr12:25245348-25245349), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [5]:
panel.merge_close_junctions()

Merging junctions within 100 bp on same chromosome...
Merged 2 junctions: KRAS_G13R_KRAS_G12D
After merging: 3 junctions remain


In [6]:
print(panel.junctions)

[Junction(KRAS_G13R_KRAS_G12D, chr12:25245348-25245350), Junction(EGFR_T790M, chr7:55181378-55181378), Junction(BRAF_V600E, chr7:140753336-140753336)]


In [7]:
fasta_file = '/Users/ctosimsen/hg38/hg38.fa'
panel.extract_design_regions_from_fasta(fasta_file, padding=200)

Extracting design regions from /Users/ctosimsen/hg38/hg38.fa with 200bp padding...
Successfully extracted 3 design regions


In [8]:
for junction in panel.junctions:
    if hasattr(junction, 'design_region'):
        print(f"\n{junction.name}:")
        print(f"  Region: {junction.design_region} ")



KRAS_G13R_KRAS_G12D:
  Region: TTCAGATAACTTAACTTTCAGCATAATTATCTTGTAATAAGTACTCATGAAAATGGTCAGAGAAACCTTTATCTGTATCAAAGAATGGTCCTGCACCAGTAATATGCATATTAAAACAAGATTTACCTCTATTGTTGGATCATATTCGTCCACAAAATGATTCTGAATTAGCTGTATCGTCAAGGCACTCTTGCCTACGCCACCAGCTCCAACTACCACAAGTTTATATTCAGTCATTTTCAGCAGGCCTTATAATAAAAATAATGAAAATGTGACTATATTAGAACATGTCACACATAAGGTTAATACACTATCAAATACTCCACCAGTACCTTTTAATACAAACTCACCTTTATATGAAAAATTATTTCAAAATACCTTACAAAATTCAATCATGAAAATT 

EGFR_T790M:
  Region: TACGTATTTTGAAACTCAAGATCGCATTCATGCGTCTTCACCTGGAAGGGGTCCATGTGCCCCTCCTTCTGGCCACCATGCGAAGCCACACTGACGTGCCTCTCCCTCCCTCCAGGAAGCCTACGTGATGGCCAGCGTGGACAACCCCCACGTGTGCCGCCTGCTGGGCATCTGCCTCACCTCCACCGTGCAGCTCATCACGCAGCTCATGCCCTTCGGCTGCCTCCTGGACTATGTCCGGGAACACAAAGACAATATTGGCTCCCAGTACCTGCTCAACTGGTGTGTGCAGATCGCAAAGGTAATCAGGGAAGGGAGATACGGGGAGGGGAGATAAGGAGCCAGGATCCTCACATGCGGTCTGCGCTCCTGGGATAGCAAGAGTTTGCCATGGGGATATG 

BRAF_V600E:
  Region: AAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCA

In [9]:
print(vars(panel.junctions[2]))

{'name': 'BRAF_V600E', 'chrom': 'chr7', 'five_prime': 140753336, 'three_prime': 140753336, 'design_region': 'AAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCAGTGGAAAAATAGCCTCAATTCTTACCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGATTTCACTGTAGCTAGACCAAAATCACCTATTTTTACTGTGAGGTCTTCATGAAGAAATATATCTGAGGTGTAGTAAGTAAAGGAAAACAGTAGATCTCATTTTCCTATCAGAGCAAGCATTATGAAGAGTTTAGGTAAGAGATCTAATTTCTATAATTCTGTAATATAATATTCTTTAAAACATAGTACTTCATCTTTCCTCTTA', 'design_start': 140753136, 'design_end': 140753536, 'junction_length': None, 'jmin_coordinate': None, 'jmax_coordinate': None}


In [10]:
panel.calculate_junction_coordinates_in_design_region()

Calculated junction coordinates for 3 junctions


In [11]:
print(vars(panel.junctions[2]))

{'name': 'BRAF_V600E', 'chrom': 'chr7', 'five_prime': 140753336, 'three_prime': 140753336, 'design_region': 'AAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCAGTGGAAAAATAGCCTCAATTCTTACCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGATTTCACTGTAGCTAGACCAAAATCACCTATTTTTACTGTGAGGTCTTCATGAAGAAATATATCTGAGGTGTAGTAAGTAAAGGAAAACAGTAGATCTCATTTTCCTATCAGAGCAAGCATTATGAAGAGTTTAGGTAAGAGATCTAATTTCTATAATTCTGTAATATAATATTCTTTAAAACATAGTACTTCATCTTTCCTCTTA', 'design_start': 140753136, 'design_end': 140753536, 'junction_length': 401, 'jmin_coordinate': 196, 'jmax_coordinate': 202}


In [12]:
panel.design_config

{'opt_melting_temperature': 60,
 'primer_pair_max_tm_difference': 5.0,
 'tm_threshold': 3.0,
 'primer_optimal_length': 22,
 'primer_length_range': [15, 30],
 'amplicon_gap_range': [20, 60],
 'max_amplicon_length': 100,
 'junction_padding_bases': 3,
 '5_primer_tail': 'GGACACTCTTTCCCTACACGACGCTCTTCCGATCTAAAAAAAAAAAAAAAAAAAATGGGAAAGAGTGTCC',
 '3_prime_tail': 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT',
 'primer_length_penalty': 1.0,
 'primer_complexity_penalty': 1.0,
 'amplicon_length_penalty': 1.0,
 'primer_min_gc': 30,
 'primer_max_gc': 70,
 'primer_gc_clamp': 0,
 'max_poly_x': 5,
 'PRIMER_MAX_SELF_ANY_TH': 45.0,
 'PRIMER_MAX_SELF_END_TH': 35.0,
 'PRIMER_PAIR_MAX_COMPL_ANY_TH': 45.0,
 'PRIMER_PAIR_MAX_COMPL_END_TH': 35.0,
 'PRIMER_MAX_HAIRPIN_TH': 24.0,
 'PRIMER_MAX_TEMPLATE_MISPRIMING_TH': 40.0,
 'PRIMER_PAIR_MAX_TEMPLATE_MISPRIMING_TH': 70.0}

In [13]:
panel.pcr_config

{'annealing_temperature': 60,
 'primer_concentration': 50,
 'dntp_concentration': 0.6,
 'dna_concentration': 50,
 'mv_concentration': 50,
 'dv_concentration': 1.5,
 'dmso_concentration': 0.0,
 'dmso_fact': 0.6,
 'formamide_concentration': 0.8,
 'salt_correction_formula': 'santalucia1998',
 'thermodynamic_table': 'santalucia1998'}

In [14]:
min_product_length = 2 * panel.design_config['primer_length_range'][0] + junction.jmax_coordinate - junction.jmin_coordinate
max_product_length = panel.design_config['max_amplicon_length']


# Set global design arguments
global_args={
    'PRIMER_TM_FORMULA': 1,
    'PRIMER_SALT_CORRECTIONS': 1,
    'PRIMER_OPT_SIZE': panel.design_config['primer_optimal_length'],
    'PRIMER_MIN_SIZE': panel.design_config['primer_length_range'][0],
    'PRIMER_MAX_SIZE': panel.design_config['primer_length_range'][1],
    'PRIMER_PRODUCT_OPT_SIZE': 60,
    'PRIMER_PAIR_WT_PRODUCT_SIZE_LT': 0.5,
    'PRIMER_PAIR_WT_PRODUCT_SIZE_GT': 2.0,
    'PRIMER_OPT_TM': panel.design_config['opt_melting_temperature'],
    'PRIMER_MIN_TM': panel.design_config['opt_melting_temperature']-panel.design_config['tm_threshold'],
    'PRIMER_MAX_TM': panel.design_config['opt_melting_temperature']+panel.design_config['tm_threshold'],
    'PRIMER_SALT_MONOVALENT': panel.pcr_config['mv_concentration'],           # The millimolar (mM) concentration of monovalent salt cations (usually KCl) in the PCR.
    'PRIMER_SALT_DIVALENT': panel.pcr_config['dv_concentration'],             # The millimolar concentration of divalent salt cations (usually MgCl^(2+)) in the PCR. 
    'PRIMER_DNA_CONC': panel.pcr_config['primer_concentration'],
    'PRIMER_DNTP_CONC': panel.pcr_config['dntp_concentration'],               # Millimolar concentration of the sum of all deoxyribonucleotide triphosphates (e.g. 4x 0.2=0.8)
    'PRIMER_DMSO_CONC': panel.pcr_config['dmso_concentration'],
    'PRIMER_FORMAMIDE_CONC': panel.pcr_config['formamide_concentration'],
    'PRIMER_NUM_RETURN': 10,                                                   # The maximum number of primer (pairs) to return.
    'PRIMER_OPT_GC_PERCENT': 50.0,                                             # Optimum GC percent. This parameter influences primer selection only if PRIMER_WT_GC_PERCENT_GT or PRIMER_WT_GC_PERCENT_LT are non-0.
    'PRIMER_MIN_GC': panel.design_config['primer_min_gc'],
    'PRIMER_MAX_GC': panel.design_config['primer_max_gc'],
    'PRIMER_MAX_END_GC': 4,
    'PRIMER_MAX_END_STABILITY': 9,
    'PRIMER_MAX_POLY_X': panel.design_config['max_poly_x'],
    'PRIMER_MAX_NS_ACCEPTED': 0,
    'PRIMER_PAIR_MAX_DIFF_TM': panel.design_config['primer_pair_max_tm_difference'],
    'PRIMER_MAX_SELF_ANY': 8.0,
    'PRIMER_MAX_SELF_END': 3.0,
    'PRIMER_PAIR_MAX_COMPL_ANY': 8.0,
    'PRIMER_PAIR_MAX_COMPL_END': 8.0,
    'PRIMER_MAX_SELF_ANY_TH': 45.0,
    'PRIMER_MAX_SELF_END_TH': 35.0,
    'PRIMER_PAIR_MAX_COMPL_ANY_TH': 45.0,
    'PRIMER_PAIR_MAX_COMPL_END_TH': 35.0,
    'PRIMER_MAX_HAIRPIN_TH': 24.0,
    'PRIMER_MAX_TEMPLATE_MISPRIMING_TH': 40.0,
    'PRIMER_PAIR_MAX_TEMPLATE_MISPRIMING_TH': 70.0,
    'PRIMER_WT_SIZE_LT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers shorter than PRIMER_OPT_SIZE.
    'PRIMER_WT_SIZE_GT': panel.design_config['primer_length_penalty'],       # Penalty weight for primers longer than PRIMER_OPT_SIZE.
    'PRIMER_WT_GC_PERCENT_LT': 0.0,
    'PRIMER_WT_GC_PERCENT_GT': 0.0,
    'PRIMER_WT_SELF_ANY': 5.0,
    'PRIMER_WT_SELF_ANY_TH': 5.0,
    'PRIMER_WT_HAIRPIN_TH': 1.0,
    'PRIMER_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_PAIR_WT_COMPL_ANY': 5.0,
    'PRIMER_PAIR_WT_COMPL_ANY_TH': 5.0,
    'PRIMER_PAIR_WT_COMPL_END_TH': 5.0,
    'PRIMER_PAIR_WT_TEMPLATE_MISPRIMING_TH': 1.0,
    'PRIMER_THERMODYNAMIC_OLIGO_ALIGNMENT': 0,                               # If set to 1, use thermodynamic values (parameters ending in "_TH")
    'PRIMER_THERMODYNAMIC_TEMPLATE_ALIGNMENT': 0,
    'PRIMER_PRODUCT_SIZE_RANGE': [[min_product_length, max_product_length]]
}


In [15]:
results_list = []


for junction in panel.junctions:
    # Set the sequence arguments
    seq_args={
        'SEQUENCE_ID': junction.name,
        'SEQUENCE_TEMPLATE': junction.design_region,
        'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[1,junction.jmin_coordinate,junction.jmax_coordinate,junction.junction_length-junction.jmax_coordinate]]
    }

    #print(global_args)

    results = primer3.bindings.design_primers(
        seq_args=seq_args,
        global_args=global_args
    )

    # Generate tables
    tables = convert_primer_data_to_tables(results)

    results_list.append(results)

    # Display tables
    #for table_name, df in tables.items():
    #    print(f"\n=== {table_name.upper().replace('_', ' ')} ===")
    #    print(df.to_string())
    #    print()

    # Save to Excel with multiple sheets
    with pd.ExcelWriter('.out/' + junction.name + '_primer_analysis.xlsx', engine='openpyxl') as writer:
        for sheet_name, df in tables.items():
            df.to_excel(writer, sheet_name=sheet_name, index=True)

    # Save individual CSV files
    for table_name, df in tables.items():
        df.to_csv(f'.out/{junction.name}_primer_{table_name}.csv', index=True)

    print("Tables saved to:")
    print("- primer_analysis.xlsx (multi-sheet Excel file)")
    print("- Individual CSV files for each table")

Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table
Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table
Tables saved to:
- primer_analysis.xlsx (multi-sheet Excel file)
- Individual CSV files for each table


In [16]:
print(seq_args)

{'SEQUENCE_ID': 'BRAF_V600E', 'SEQUENCE_TEMPLATE': 'AAAAAATAAGAACACTGATTTTTGTGAATACTGGGAACTATGAAAATACTATAGTTGAGACCTTCAATGACTTTCTAGTAACTCAGCAGCATCTCAGGGCCAAAAATTTAATCAGTGGAAAAATAGCCTCAATTCTTACCATCCACAAAATGGATCCAGACAACTGTTCAAACTGATGGGACCCACTCCATCGAGATTTCACTGTAGCTAGACCAAAATCACCTATTTTTACTGTGAGGTCTTCATGAAGAAATATATCTGAGGTGTAGTAAGTAAAGGAAAACAGTAGATCTCATTTTCCTATCAGAGCAAGCATTATGAAGAGTTTAGGTAAGAGATCTAATTTCTATAATTCTGTAATATAATATTCTTTAAAACATAGTACTTCATCTTTCCTCTTA', 'SEQUENCE_PRIMER_PAIR_OK_REGION_LIST': [[1, 196, 202, 199]]}


In [17]:
print(results_list[1])

{'PRIMER_LEFT_EXPLAIN': 'considered 5832, GC content failed 282, low tm 117, high tm 398, high end compl 82, not in any ok left region 4904, ok 47', 'PRIMER_RIGHT_EXPLAIN': 'considered 5739, GC content failed 40, low tm 330, high tm 237, high any compl 16, high end compl 98, not in any ok right region 4904, ok 113', 'PRIMER_PAIR_EXPLAIN': 'considered 3185, unacceptable product size 1672, tm diff too large 25, ok 1488', 'PRIMER_LEFT_NUM_RETURNED': 10, 'PRIMER_RIGHT_NUM_RETURNED': 10, 'PRIMER_INTERNAL_NUM_RETURNED': 0, 'PRIMER_PAIR_NUM_RETURNED': 10, 'PRIMER_PAIR': [{'PENALTY': 59.634763162670815, 'COMPL_ANY': 4.0, 'COMPL_END': 2.0, 'PRODUCT_SIZE': 53, 'PRODUCT_TM': 80.58865126231156}, {'PENALTY': 59.87055189528005, 'COMPL_ANY': 4.0, 'COMPL_END': 3.0, 'PRODUCT_SIZE': 55, 'PRODUCT_TM': 81.56789517311775}, {'PENALTY': 60.277200224035475, 'COMPL_ANY': 4.0, 'COMPL_END': 2.0, 'PRODUCT_SIZE': 55, 'PRODUCT_TM': 81.56789517311775}, {'PENALTY': 60.37055189528005, 'COMPL_ANY': 4.0, 'COMPL_END': 2.

In [29]:
display(create_primer_dataframe(results_list[2]))

,pair_id,pair_penalty,product_size,product_tm,compl_any_th,compl_end_th,left_sequence,left_coords,left_tm,left_gc_percent,...,left_end_stability,right_sequence,right_coords,right_tm,right_gc_percent,right_penalty,right_self_any_th,right_self_end_th,right_hairpin_th,right_end_stability
0,0,100.592708,62,73.823562,None,None,TGATGGGACCCACTCCATCG,"[173, 20]",59.250824,60.000000,...,3.84,ACAGTAAAAATAGGTGATTTTGGTCTAGC,"[234, 29]",58.156468,34.482759,38.843532,None,None,None,3.42
1,1,101.011634,63,74.358392,None,None,CTGATGGGACCCACTCCATCG,"[172, 21]",60.168102,61.904762,...,3.84,ACAGTAAAAATAGGTGATTTTGGTCTAGC,"[234, 29]",58.156468,34.482759,38.843532,None,None,None,3.42
2,2,101.366290,61,73.949269,None,None,GATGGGACCCACTCCATCG,"[174, 19]",57.477242,63.157895,...,3.84,ACAGTAAAAATAGGTGATTTTGGTCTAGC,"[234, 29]",58.156468,34.482759,38.843532,None,None,None,3.42
3,3,102.652413,66,75.238938,None,None,TGATGGGACCCACTCCATCG,"[173, 20]",59.250824,60.000000,...,3.84,CCTCACAGTAAAAATAGGTGATTTTGGT,"[238, 28]",58.096763,35.714286,37.903237,None,None,None,3.67
4,4,102.710586,63,74.358392,None,None,TGATGGGACCCACTCCATCG,"[173, 20]",59.250824,60.000000,...,3.84,CACAGTAAAAATAGGTGATTTTGGTCTAGC,"[235, 30]",59.038590,36.666667,38.961410,None,None,None,3.42
5,5,102.743733,71,75.764394,None,None,TGATGGGACCCACTCCATCG,"[173, 20]",59.250824,60.000000,...,3.84,GAAGACCTCACAGTAAAAATAGGTGAT,"[243, 27]",57.005443,37.037037,27.994557,None,None,None,3.06
6,6,103.071339,67,75.720713,None,None,CTGATGGGACCCACTCCATCG,"[172, 21]",60.168102,61.904762,...,3.84,CCTCACAGTAAAAATAGGTGATTTTGGT,"[238, 28]",58.096763,35.714286,37.903237,None,None,None,3.67
7,7,103.129512,64,74.876509,None,None,CTGATGGGACCCACTCCATCG,"[172, 21]",60.168102,61.904762,...,3.84,CACAGTAAAAATAGGTGATTTTGGTCTAGC,"[235, 30]",59.038590,36.666667,38.961410,None,None,None,3.42
8,8,103.162659,72,76.205414,None,None,CTGATGGGACCCACTCCATCG,"[172, 21]",60.168102,61.904762,...,3.84,GAAGACCTCACAGTAAAAATAGGTGAT,"[243, 27]",57.005443,37.037037,27.994557,None,None,None,3.06
9,9,103.277758,71,75.764394,None,None,TGATGGGACCCACTCCATCG,"[173, 20]",59.250824,60.000000,...,3.84,GAAGACCTCACAGTAAAAATAGGTGATT,"[243, 28]",57.471418,35.714286,28.528582,None,None,None,2.57


In [ ]:
oligo_calc = primer3.thermoanalysis.ThermoAnalysis()

oligo_calc.set_thermo_args(
    mv_conc = 50.0,
    dv_conc = 1.5,
    dntp_conc = 0.6,
    dna_conc = 50.0,
    dmso_conc = 0.0,
    dmso_fact = 0.6,
    formamide_conc = 0.0,
    annealing_temp_c = -10,
    temp_c = 37.0,
    max_nn_length = 60,
    max_loop = 30,
    tm_method = 'santalucia',
    salt_corrections_method = 'santalucia'
)

oligo_calc.calc_hairpin('TGCTTGCTCTGATAGGAAAATGA')

62.694057579500395

In [24]:
primer_list = ['TGATGGGACCCACTCCATCG','ACAGTAAAAATAGGTGATTTTGGTCTAGC', 'CTGATGGGACCCACTCCATCG', 'ACAGTAAAAATAGGTGATTTTGGTCTAGC']

In [25]:
for primer in primer_list:
    print(oligo_calc.calc_tm(primer))  # Print the melting temp

61.337383679693005
60.335502939154594
62.24775924663061
60.335502939154594


In [26]:
from itertools import combinations

for primer in primer_list:
    print(oligo_calc.calc_tm(primer))  # Print the melting temp
    print(oligo_calc.calc_homodimer(primer))
    print(oligo_calc.calc_hairpin(primer))

for a, b in combinations(primer_list, 2):
    print(f"Primer 1: {a} + Primer 2: {b}")
    print(oligo_calc.calc_heterodimer(a, b))
    print(oligo_calc.calc_end_stability(a, b))



61.337383679693005
ThermoResult(structure_found=True, tm=17.78, dg=-7662.21, dh=-53800.00, ds=-148.76)
ThermoResult(structure_found=True, tm=53.81, dg=-2127.99, dh=-41400.00, ds=-126.62)
60.335502939154594
ThermoResult(structure_found=True, tm=-3.59, dg=-1277.61, dh=-66000.00, ds=-208.68)
ThermoResult(structure_found=True, tm=45.21, dg=-786.81, dh=-30500.00, ds=-95.80)
62.24775924663061
ThermoResult(structure_found=True, tm=17.78, dg=-7662.21, dh=-53800.00, ds=-148.76)
ThermoResult(structure_found=True, tm=53.81, dg=-2127.99, dh=-41400.00, ds=-126.62)
60.335502939154594
ThermoResult(structure_found=True, tm=-3.59, dg=-1277.61, dh=-66000.00, ds=-208.68)
ThermoResult(structure_found=True, tm=45.21, dg=-786.81, dh=-30500.00, ds=-95.80)
Primer 1: TGATGGGACCCACTCCATCG + Primer 2: ACAGTAAAAATAGGTGATTTTGGTCTAGC
ThermoResult(structure_found=True, tm=-9.42, dg=-4000.04, dh=-41000.00, ds=-119.30)
ThermoResult(structure_found=True, tm=-90.00, dg=-120.32, dh=-16000.00, ds=-51.20)
Primer 1: TGATGGG